In [31]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
import sklearn.model_selection as model_selection
import contextily as cx
import geopandas as gpd
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from scipy import stats
import optuna
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import geopandas as gpd

gpd.io.file.fiona.drvsupport.supported_drivers["KML"] = "rw"

In [32]:
NFOLDS = 5
SEED = 42

In [33]:
def get_oof(clf, kf, x_train, y_train, x_test):
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train, y_train)):
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)
    
def rmlse(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

In [34]:
# import data 

# Read the apartment datasets
apartments_train = pd.read_csv("resources/data/apartments_train.csv").set_index("id")
apartments_train["split"] = "train"
apartments_test = pd.read_csv("resources/data/apartments_test.csv").set_index("id")
apartments_test["split"] = "test"

# Create a DataFrame of all apartments
apartments = pd.concat([apartments_train, apartments_test])

# Read the building datasets
buildings_train = pd.read_csv("resources/data/buildings_train.csv").set_index("id")
buildings_train["split"] = "train"
buildings_test = pd.read_csv("resources/data/buildings_test.csv").set_index("id")
buildings_test["split"] = "test"

# Create a GeoDataFrame of all buildings
buildings = pd.concat([buildings_train, buildings_test])
buildings = gpd.GeoDataFrame(buildings, geometry=gpd.points_from_xy(
    buildings.longitude, buildings.latitude, crs="EPSG:4326"
))

In [35]:
# Find all buildings missing coordinates
no_coords = buildings.latitude.isna() | buildings.longitude.isna()
street = buildings[~no_coords & (buildings.street == "пос. Коммунарка")]

buildings.loc[no_coords, "latitude"] = street.latitude.mean()
buildings.loc[no_coords, "longitude"] = street.longitude.mean()
buildings.loc[no_coords, "district"] = street.district.mode()[0]

In [36]:
# The coordinates of the southwest and northeast corners of a rectangle approximately encompassing Moscow
MOSCOW_SW_LAT = 55
MOSCOW_SW_LON = 35
MOSCOW_NE_LAT = 57
MOSCOW_NE_LON = 39

In [37]:
# Find all buildings with coordinates outside of Moscow
outside = (((buildings.latitude < MOSCOW_SW_LAT) | (buildings.latitude > MOSCOW_NE_LAT)
          | (buildings.longitude < MOSCOW_SW_LON) | (buildings.longitude > MOSCOW_NE_LON)))
buildings[outside][["split", "latitude", "longitude", "district", "street", "address", "constructed", "material", "stories"]]

for idx, building in buildings[outside].iterrows():
    street = buildings[~outside & (buildings.street == building.street)]
    if len(street):
        buildings.loc[idx, "latitude"] = street.latitude.mean()
        buildings.loc[idx, "longitude"] = street.longitude.mean()
        buildings.loc[idx, "district"] = street.district.mode()[0]
    else:
        buildings.loc[idx, "latitude"] = buildings[~outside].latitude.mean()
        buildings.loc[idx, "longitude"] = buildings[~outside].longitude.mean()

In [38]:
# Impute district
no_district = buildings.district.isna()
districts = buildings.loc[no_district].apply(
    lambda b: buildings.loc[
        (buildings[~no_district][["latitude", "longitude"]] - b[["latitude", "longitude"]]).abs().sum(axis=1).idxmin()
    ].district,
    axis=1
)
districts.rename("district", inplace=True)
buildings.update(districts)


In [39]:
stations = gpd.read_file("resources/metro_stations.kml", driver="KML").drop(columns=["Description"]).rename(columns={"Name": "name"})

# The Earth's radius in meters
EARTH_RADIUS = 6371000

# Create temporary columns for coordinates given in radians
stations["lon_rad"] = np.radians(stations.geometry.x)
stations["lat_rad"] = np.radians(stations.geometry.y)
buildings["lon_rad"] = np.radians(buildings.longitude)
buildings["lat_rad"] = np.radians(buildings.latitude)

# Calculate the distance to the nearest metro station for each building using
# the haversine formula with the Earth's radius as given above
buildings["metro_distance"] = buildings.apply(
    lambda row: np.min(
        2
        * EARTH_RADIUS
        * np.arcsin(
            np.sqrt(
                np.sin((stations.lat_rad - row.lat_rad) / 2) ** 2
                + np.cos(row.lat_rad)
                * np.cos(stations.lat_rad)
                * np.sin((stations.lon_rad - row.lon_rad) / 2) ** 2
            )
        )
    ),
    axis=1,
)


In [40]:
# Read park and garden location data
parks = gpd.read_file("resources/parks_and_gardens.kml", driver="KML").drop(columns=["Description"]).rename(columns={"Name": "name"})

# Create columns for coordinates given in radians
parks["lon_rad"] = np.radians(parks.geometry.x)
parks["lat_rad"] = np.radians(parks.geometry.y)

# Calculate the distance to the nearest park or garden for each building using
# the haversine formula with the Earth's radius as given above
buildings["park_distance"] = buildings.apply(
    lambda row: np.min(
        2
        * EARTH_RADIUS
        * np.arcsin(
            np.sqrt(
                np.sin((parks.lat_rad - row.lat_rad) / 2) ** 2
                + np.cos(row.lat_rad)
                * np.cos(parks.lat_rad)
                * np.sin((parks.lon_rad - row.lon_rad) / 2) ** 2
            )
        )
    ),
    axis=1,
)


In [41]:
# Read square location data
squares = gpd.read_file("resources/squares.kml", driver="KML").drop(columns=["Description"]).rename(columns={"Name": "name"})

# Create columns for coordinates given in radians
squares["lon_rad"] = np.radians(squares.geometry.x)
squares["lat_rad"] = np.radians(squares.geometry.y)

# Calculate the distance to the nearest square for each building using the
# haversine formula with the Earth's radius as given above
buildings["square_distance"] = buildings.apply(
    lambda row: np.min(
        2
        * EARTH_RADIUS
        * np.arcsin(
            np.sqrt(
                np.sin((squares.lat_rad - row.lat_rad) / 2) ** 2
                + np.cos(row.lat_rad)
                * np.cos(squares.lat_rad)
                * np.sin((squares.lon_rad - row.lon_rad) / 2) ** 2
            )
        )
    ),
    axis=1,
)

# Drop the temporary radian columns
stations.drop(columns=["lon_rad", "lat_rad"], inplace=True)
buildings.drop(columns=["lon_rad", "lat_rad"], inplace=True)

In [42]:
buildings_train = buildings.loc[buildings["split"] == "train", :]
buildings_train.drop(columns = "split", inplace = True)
apartments = pd.read_csv('resources/data/apartments_train.csv')
data = pd.merge(apartments, buildings_train, how='left', left_on='building_id', right_index=True)

buildings_test = buildings.loc[buildings["split"] == "test", :]
buildings_test.drop(columns = "split", inplace = True)
apartments_test = pd.read_csv('resources/data/apartments_test.csv')
data_test = pd.merge(apartments_test, buildings_test, how='left', left_on='building_id', right_index=True)

/Users/jimtotland/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
# feature ballog
data['ballog'] = data['loggias'] + data['balconies']
data_test['ballog'] = data_test['loggias'] + data_test['balconies']

In [44]:
data['bathrooms_total'] = data['bathrooms_private'] + data['bathrooms_shared']
data_test['bathrooms_total'] = data_test['bathrooms_private'] + data_test['bathrooms_shared']

In [45]:
data['area_per_room'] = data['area_total']/data['rooms']
data_test['area_per_room'] = data_test['area_total']/data_test['rooms']

In [52]:
NUMERIC_FEATURES = ["latitude", "longitude", "constructed", "area_total",
            "rooms", "ballog", "metro_distance", "park_distance",
            "square_distance", "stories", "floor", "ceiling", "bathrooms_total"]
CATEGORICAL_FEATURES = ["district", "material", "condition", "heating", "new", 
                "layout", "parking", "garbage_chute",]
data[CATEGORICAL_FEATURES] = data[CATEGORICAL_FEATURES].astype('category')
#data_test[CATEGORICAL_FEATURES] = data_test[CATEGORICAL_FEATURES].astype('category')
FEATURES = CATEGORICAL_FEATURES + NUMERIC_FEATURES

In [53]:
lgb_params = {
    'num_leaves': 10,
    'max_depth': 5, 
    'random_state':SEED, 
    'silent' : True, 
    'metric': 'mse',
    'n_jobs': 4, 
    'n_estimators': 2000,
    'colsample_bytree': 0.95,
    'subsample': 0.9,
    'learning_rate': 0.05
}

cb_params = {
    'n_estimators': 1000,
    'learning_rate': 0.01,
    'thread_count': -1,
    'depth': 7,
    'silent': True,
    'random_seed': SEED,
    'bagging_temperature': 0.2
}

In [ ]:
from scipy.stats import skew

skewed_feats = data[NUMERIC_FEATURES].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
print(skewed_feats)

df_scaled = data.copy()
df_scaled[skewed_feats] = np.log1p(df_scaled[skewed_feats])

df_scaled_test = data_test.copy()
df_scaled_test[skewed_feats] = np.log1p(df_scaled_test[skewed_feats])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(data[FEATURES], data.price, random_state = SEED, test_size = 0.20, stratify=round(np.log(data.price)))

In [55]:
X_train.dtypes

district           float64
material           float64
condition          float64
heating            float64
new                float64
layout             float64
parking            float64
garbage_chute      float64
latitude           float64
longitude          float64
constructed        float64
area_total         float64
rooms              float64
ballog             float64
metro_distance     float64
park_distance      float64
square_distance    float64
stories            float64
floor              float64
ceiling            float64
bathrooms_total    float64
dtype: object

In [56]:
model1 = lgb.LGBMRegressor(**lgb_params)
model2 = CatBoostRegressor(**cb_params)

In [57]:
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED) 

cb_oof_train, cb_oof_test = get_oof(model2, kf, X_train, y_train, X_test)
lgbm_oof_train, lgbm_oof_test = get_oof(model1, kf, X_train, y_train, X_test)

In [58]:
x_train = np.concatenate((
        lgbm_oof_train,
        cb_oof_train,
    ), axis=1)

x_test = np.concatenate((
    lgbm_oof_test,
    cb_oof_test,
), axis=1)

In [59]:
# XGB meta model.
param = {
        'max_depth': 3, 
        'reg_alpha': 0.0012, 'reg_lambda': 0.003, 
        'min_child_weight': 0, 'gamma': 2, 
        'learning_rate': 0.0132, 'colsample_bytree': 0.45
        }

dtrain = xgb.DMatrix(x_train, label= y_train)
META_MODEL = xgb.train(param, dtrain, num_boost_round=2000)

In [60]:
dtest = xgb.DMatrix(x_test)
preds = META_MODEL.predict(dtest)
#rmlse(y_test, preds)
for i in range(len(preds)):
    if preds[i] < 0:
        preds[i] = 0
rmlse(y_test, preds)

0.23223219354876223